In [5]:
# Menginstal library jika belum diinstal
!pip install google-play-scraper

In [15]:
from google_play_scraper import app, reviews, Sort, reviews_all

Opsi yang terdapat pada fungsi `reviews`:
1. ID aplikasi yang ada pada Google Play, didapat dari link aplikasi pada Google Play setelah sama dengan(=).
Contoh https://play.google.com/store/apps/details?id=com.ea.game.pvzfree_row maka ID aplikasinya adalah com.ea.game.pvzfree_row
2. `lang`: Bahasa yang digunakan dalam ulasan.
[Reverensi singkatan nama negara]('https://www.iban.com/country-codes')
3. `country`: Asal negara ulasan.
[Reverensi singkatan nama negara]('https://www.iban.com/country-codes')
4. `sort`: Mode pengurutan ulasan. Bisa menggunakan `Sort.MOST_RELEVANT` (Paling relevan) atau `Sort.NEWEST` (Paling baru).
5. `count`: Jumlah ulasan yang ingin diambil. Jika menggunakan script `reviews_all`, maka akan mengambil semua ulasan yang ada dan mengabaikan jumlah `count`.
6. `filter_score_with`: Rating ulasan (1 sd 5), artinya apabila diisi 1 maka ulasan yang diambil hanya ulasan pada rating 1, atau bisa diisi None untuk mengambil ulasan pada semua rating.

source : https://pypi.org/project/google-play-scraper/

In [16]:
detail_app = app(
    'com.ea.game.pvzfree_row',
    lang='id',
    country='id'
)
detail_app

{'title': 'Plants vs. Zombies™',
 'description': 'Bersiaplah untuk memupuk tanamanmu karena segerombolan zombi pencinta keseruan akan menyerbu rumahmu. Gunakan persenjataan yang berisi 49 tanaman pembunuh zombi — peashooter, wall-nut, cherry bomb, dan banyak lagi — untuk membinasakan 26 jenis zombi sebelum mereka mendobrak pintumu. \r\n\r\nAplikasi ini menawarkan pembelian dalam aplikasi. Kamu bisa menonaktifkan pembelian dalam aplikasi menggunakan pengaturan perangkatmu\r\n\r\nPEMENANG LEBIH DARI 30 PENGHARGAAN GAME TAHUNAN TERBAIK*\r\n\r\n50 LEVEL SERU ABIS\r\nTaklukkan ke-50 level mode Petualangan — siang, malam, kabut, di kolam renang, di atap, dan banyak lagi. Plus, tangkis gelombang zombi berkelanjutan selama mungkin dengan mode Survival!\r\n\r\nBUKAN SETAN KUBUR BIASA\r\nBertempur melawan zombi pelompat galah, zombi snorkeling, kepala ember, dan 26 zombi seru lainnya. Masing-masing memiliki keahlian spesial sendiri, jadi kamu harus berpikir cepat dan menanam lebih cepat untuk me

In [17]:
scrap_app, continuation_token = reviews(
    'com.ea.game.pvzfree_row',
    lang='id',
    country='id',
    sort=Sort.NEWEST,
    count = 10000,
    filter_score_with = None
)

**Menyimpan Ulasan dalam file CSV atau JSON**

Sebagaimana diketahui Pandas merupakan paket python yang memberikan kemudahan dalam memproses dan menganalisa data. Biasanya data-data yang telah diolah disimpan dalam file, dan yang paling umum adalah file dengan format csv atau (comma separated value).

Untuk melakukan export Pandas DataFrame ke file csv, Pandas memiliki berbagai cara, diantaranya dengan memanfaatkan library `csv` atau menggunakan fungsi `.to_csv`.

source:

https://hub.idbigdata.com/sigit-prasetyo/export-pandas-dataframe-sebagai-file-csv--47

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html

In [18]:
# Menyimpan data dengan library csv

import csv
with open('ulasan_aplikasi.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Review'])  # Menulis header kolom
    for review in scrap_app:
        writer.writerow([review['content']])  # Menulis konten ulasan ke dalam file CSV

In [19]:
# Menyimpan data dengan fungsi .to_csv
import pandas as pd

scrap_df = pd.DataFrame(scrap_app)
scrap_df.to_csv('ulasan_apk.csv',# Nama file yang akan disimpan (diakhiri dengan .csv)
                index = False,   # Apakah index data akan ikut dimasukan kedalam file csv?
                sep = ',',       # Tanda yang memisahkan antar kolom (default: ,)
                header = True,   # Apakah header atau nama kolom disertakan pada file csv?
                mode = 'w'       # Mode pembuatan file memotong jika sudah ada
                )

In [21]:
# Menyimpan data dengan fungsi .to_json

scrap_df.to_json('ulasan_apk.json')